In [19]:
import numpy as np
from collections import Counter
from nltk import ngrams

## BLEU Score(Bilingual Evaluation Understudy Score)

`언어 모델(Language Model)의 성능 측정을 위한 평가 방법`으로 `펄플렉서티(perplexity, PPL)`가 있음

기계 번역기에도 PPL을 평가에 사용할 수는 있지만, PPL은 번역의 성능을 직접적으로 반영하는 수치라 보기엔 어려움이 있음

기계 번역의 성능이 얼마나 뛰어난가를 측정하기 위해 사용되는 대표적인 방법으로 `BLEU(Bilingual Evaluation Understudy)`가 있음

BLEU는 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하여 번역에 대한 성능을 측정하는 방법

측정 기준은 n-gram에 기반

BLEU는 완벽한 방법이라고는 할 수는 없지만 몇 가지 이점이 있음

언어에 구애받지 않고 계산속도가 빠르다는 것

BLEU는 PPL과는 달리 높을 수록 성능이 더 좋음을 의미

### 단어 개수 카운트로 측정(Unigram Precision)

한국어-영어 번역기의 성능을 측정한다고 가정

두 개의 기계 번역기가 존재하고 두 기계 번역기에 같은 한국어 문장을 입력하여 번역된 영어 문장의 성능을 측정

번역된 문장을 각각 Candidate1, 2 라고 할때,

이 문장의 성능을 평가하기 위해서는 정답으로 비교되는 문장이 필요

세 명의 사람에게 한국어를 보고 영작해보라고 하여 세 개의 번역 문장 Reference1, 2, 3 을 구했다고 가정

__Example 1__
- Candidate1 : It is a guide to action which ensures that the military always obeys the commands of the party.
- Candidate2 : It is to insure the troops forever hearing the activity guidebook that party direct.
- Reference1 : It is a guide to action that ensures that the military will forever heed Party commands.
- Reference2 : It is the guiding principle which guarantees the military forces always being under the command of the Party.
- Reference3 : It is the practical guide for the army always to heed the directions of the party.

Ca 1, 2를 Ref 1, 2, 3과 비교하여 성능을 측정

가장 직관적인 성능 평가 방법은 Ref 1, 2, 3 중 어느 한 문장이라도 등장한 단어의 개수를 Ca에서 세는 것

그 후에 Ca의 모든 단어의 카운트의 합, 즉 Ca에서의 총 단어의 수으로 나눠주는 것

해당 측정 방법을 `유니그램 정밀도(Unigram Precision)` 라고 하고 식으로 나타내면 다음과 같음

$ \text{Unigram Precision} = 
\frac{\text{Ref들 중에서 존재하는 Ca의 단어의 수}}{\text{Ca의 총 단어 수}} = 
\frac{\text{the number of Ca words(unigrams) which occur in any Ref}}{\text{the total number of words in the Ca}} $

Ca1의 단어들은 Ref1, Ref2, Ref3에서 전반적으로 등장하는 반면, Ca2는 그렇지 않음

이는 Ca1이 Ca2보다 더 좋은 번역 문장임을 의미

Ca1에 있는 단어 중 Ref1, Ref2, Ref3 어디에도 등장하지 않은 단어는 obeys뿐

위의 계산 방법에 따르면 Ca1과 Ca2의 유니그램 정밀도는 아래와 같음

$ \text{Ca1 Unigram Precision} = \frac{17}{18} $

$ \text{Ca2 Unigram Precision} = \frac{8}{14} $

### 중복을 제거하여 보정(Modified Unigram Precision)

__Example 2__
- Candidate : the the the the the the the
- Reference1 : the cat is on the mat
- Reference2 : there is a cat on the mat

위의 Ca는 the만 7개가 등장한 터무니 없는 번역

하지만 이 번역은 유니그램 정밀도에 따르면 $\frac{7}{7}$ 로 최고의 성능 평가를 받게 됨

이를 보정하기 위해서는 정밀도의 분자를 계산하기 위해 Ref와 매칭하며 카운트하는 과정에서 Ca의 유니그램이 이미 Ref에서 매칭된 적이 있었는지를 고려

유니그램이 하나의 Ref에서 최대 몇 번 등장했는지를 카운트,

해당 값이 기존의 단순 카운트한 값보다 작은 경우에는 이 값을 최종 카운트 값으로 대체

$ Count_{clip} = min(Count, \text{Maximum refernce count}) $

$ \text{Modified Unigram Precision} = 
\frac{\text{Ca의 각 유니그램에 대해}Count_{clip}\text{을 수행한 값의 총 합}}{\text{Ca의 총 단어 수}} = 
\frac{\sum_{unigram \in Candidate} Count_{clip}(unigram)}{\sum_{unigram \in Candidate} Count(unigram)} $

분모의 경우에는 이전과 동일하게 Ca의 모든 유니그램에 대해서 각각 카운트하고 모두 합한 값 사용

Ca의 기존 유니그램 정밀도는 $\frac{7}{7}$ 이였으나 보정된 유니그램 정밀도는 $\frac{2}{7}$ 와 같이 변경

In [20]:
# 토큰화 된 문장(tokens)에서 n-gram을 카운트
def simple_count(tokens, n):
    return Counter(ngrams(tokens, n))

In [21]:
# Example 1의 Ca1
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."

In [22]:
tokens = candidate.split() # 토큰화
result = simple_count(tokens, 1) # n = 1은 유니그램
print('유니그램 카운트 :',result)

유니그램 카운트 : Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [23]:
# Example 2의 Ca
candidate = 'the the the the the the the'

In [24]:
tokens = candidate.split() # 토큰화
result = simple_count(tokens, 1) # n = 1은 유니그램
print('유니그램 카운트 :',result)

유니그램 카운트 : Counter({('the',): 7})


In [25]:
def count_clip(candidate, reference_list, n):
    # Ca 문장에서 n-gram 카운트
    ca_cnt = simple_count(candidate, n)
    max_ref_cnt_dict = dict()

    for ref in reference_list: 
        # Ref 문장에서 n-gram 카운트
        ref_cnt = simple_count(ref, n)

        # 각 Ref 문장에 대해서 비교하여 n-gram의 최대 등장 횟수를 계산.
        for n_gram in ref_cnt: 
            if n_gram in max_ref_cnt_dict:
                max_ref_cnt_dict[n_gram] = max(ref_cnt[n_gram], max_ref_cnt_dict[n_gram])
            else:
                max_ref_cnt_dict[n_gram] = ref_cnt[n_gram]

    return {
            # count_clip = min(count, max_ref_count)
            n_gram: min(ca_cnt.get(n_gram, 0), max_ref_cnt_dict.get(n_gram, 0)) for n_gram in ca_cnt
        }

In [26]:
# Example 2
candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]

In [27]:
result = count_clip(candidate.split(),list(map(lambda ref: ref.split(), references)),1)
print('보정된 유니그램 카운트 :',result)

보정된 유니그램 카운트 : {('the',): 2}


In [28]:
# 보정된 정밀도 연산 함수
def modified_precision(candidate, reference_list, n):
    clip_cnt = count_clip(candidate, reference_list, n) 
    total_clip_cnt = sum(clip_cnt.values()) # 분자

    cnt = simple_count(candidate, n)
    total_cnt = sum(cnt.values()) # 분모

    # 분모가 0이 되는 것을 방지
    if total_cnt == 0: 
        total_cnt = 1

    # 분자 : count_clip의 합, 분모 : 단순 count의 합 ==> 보정된 정밀도
    return (total_clip_cnt / total_cnt)

In [29]:
result = modified_precision(candidate.split(), list(map(lambda ref: ref.split(), references)), n=1)
print('보정된 유니그램 정밀도 :', result) # 2/7 = 0.285714

보정된 유니그램 정밀도 : 0.2857142857142857


### 순서를 고려하기 위해서 n-gram으로 확장

BoW 표현과 유사하게, 유니그램 정밀도와 같이 각 단어의 빈도수로 접근하는 방법은 결국 단어의 순서를 고려하지 않는다는 특징

Example 1에 Ca3이라는 새로운 문장을 추가하여, 기존의 Ca1과 비교

__Example 1__
- Candidate1 : It is a guide to action which ensures that the military always obeys the commands of the party.
- Candidate2 : It is to insure the troops forever hearing the activity guidebook that party direct.
- Candidate3 : the that military a is It guide ensures which to commands the of action obeys always party the.
- Reference1 : It is a guide to action that ensures that the military will forever heed Party commands.
- Reference2 : It is the guiding principle which guarantees the military forces always being under the command of the Party.
- Reference3 : It is the practical guide for the army always to heed the directions of the party.

Ca3은 사실 Ca1에서 모든 유니그램의 순서를 랜덤으로 섞은 실제 영어 문법에 맞지 않은 문장

하지만 Ref 1, 2, 3과 비교하여 유니그램 정밀도를 적용하면 Ca1과 Ca3의 두 정밀도는 동일

유니그램 정밀도는 유니그램의 순서를 전혀 고려하지 않기 때문

이를 위한 대안으로 개별적인 유니그램/단어로서 카운트하는 유니그램 정밀도에서 

다음에 등장한 단어까지 함께 고려하여 카운트하도록 유니그램 외에도 Bigram, Trigram, 4-gram 단위 등으로 계산한 정밀도. 즉, n-gram을 이용한 정밀도를 도입

이들 각각은 카운트 단위를 2개, 3개, 4개로 보느냐의 차이로 2-gram Precision, 3-gram Precision, 4-gram Precision이라고 하기도 함

$ p_1 = \frac{\sum_{unigram \in Candidate} Count_{clip}(unigram)}{\sum_{unigram \in Candidate} Count(unigram)} $

$ p_n = \frac{\sum_{n-gram \in Candidate} Count_{clip}(n-gram)}{\sum_{n-gram \in Candidate} Count(n-gram)} $

BLEU는 보정된 정밀도 $p_1, p_2, p_3, ... , p_n$를 모두 조합하여 사용

식으로 표현하면 다음과 같음

$ BLEU = exp( \displaystyle\sum_{n=1}^{N} w_n log \, p_n) $

p_n : 각 gram의 보정된 정밀도

N : n-gram에서 n의 최대 숫자, 보통 4의 값을 사용

w_n : 각 gram의 보정된 정밀도에 서로 다른 가중치, 이 가중치의 합은 1

### 짧은 문장 길이에 대한 패널티(Brevity Penalty)

n-gram으로 단어의 순서를 고려한다고 하더라도 여전히 남아있는 문제가 있음

Ca의 길이에 BLEU의 점수가 과한 영향을 받을 수 있다는 점

Example 1에 다음의 Ca를 추가

Candidate4 : it is

해당 문장은 유니그램 정밀도나 바이그램 정밀도가 모두 1로 높은 정밀도를 가짐 

이과 같이 제대로 된 번역이 아님에도 문장의 길이가 짧다는 이유로 높은 점수를 받는 문제가 있음

따라서 Ca가 Ref보다 문장의 길이가 짧은 경우에는 점수에 패널티를 줄 필요가 있음 이를 브레버티 패널티(Brevity Penalty)라 함

__Example 3__
- Candidate 1: I always invariably perpetually do.
- Candidate 2: I always do.
- Reference 1: I always do.
- Reference 2: I invariably do.
- Reference 3: I perpetually do.

Ca1은 가장 많은 단어를 사용했지만 Ca2보다 좋지 못한 번역

Ref의 단어를 가장 많이 사용한 것이 꼭 좋은 번역이라는 의미는 아님

Ca의 길이가 불필요하게 Ref보다 긴 경우에는 BLEU 수식에서 정밀도를 n-gram으로 확장하여 바이그램, 트라이그램 정밀도 등을 모두 계산에 사용하고 있는 것만으로도 이미 패널티를 받고 있음

따라서 브레버티 패널티를 설계할 때 더 긴 경우 까지 고려할 필요는 없음

브레버티 패널티는 BLEU의 식에 곱하는 방식으로 사용, 이를 BP라 할때 식은 다음과 같음

$ BLEU = BP \times exp( \displaystyle\sum_{n=1}^{N} w_n log \, p_n) $

$ BP = \begin{cases}
        1 & if \, c > r \\
        e^{(1-r/c)} & if \, c \leq r \\
    \end{cases} $

c : Candidate의 길이

r : Candidate와 가장 길이 차이가 작은 Reference의 길이

Ref가 1개라면 Ca와 Ref의 두 문장의 길이만을 가지고 계산

Ref가 여러 개일 때를 가정하고 있으므로 r은 모든 Ref들 중에서 Ca와 가장 길이 차이가 작은 Ref의 길이

In [30]:
# Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
def closest_ref_length(candidate, reference_list):
    ca_len = len(candidate) # ca 길이
    ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    return closest_ref_len

만약 Ca와 길이가 정확히 동일한 Ref가 있다면 길이 차이가 0인 최고 수준의 매치(best match length)

또한 만약 서로 다른 길이의 Ref이지만 Ca와 길이 차이가 동일한 경우에는 더 작은 길이의 Ref를 택함

예를 들어 Ca가 길이가 10인데, Ref 1, 2가 각각 9와 11이라면 길이 차이는 동일하게 1밖에 나지 않지만 9를 택함

In [31]:
def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)

    if ca_len > ref_len:
        return 1

    # candidate가 비어있다면 BP = 0 → BLEU = 0.0
    elif ca_len == 0 :
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

수식처럼 c가 r보다 클 경우 1을 리턴, 이외의 경우에는 $e^{1-r/c}$를 리턴

In [32]:
def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights,start=1)] 
    # p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

## NLTK를 사용한 BLEU 측정

In [33]:
import nltk.translate.bleu_score as bleu

In [34]:
candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

In [35]:
print('실습 코드의 BLEU :', bleu_score(candidate.split(),list(map(lambda ref: ref.split(), references))))
print('패키지 NLTK의 BLEU :', bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

실습 코드의 BLEU : 0.5045666840058485
패키지 NLTK의 BLEU : 0.5045666840058485
